In [1]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text, MetaData, Table

db_url = URL.create(
    drivername="mysql+pymysql",
    username="teamx",
    password="#C!D123^-c12",
    host="112.125.88.107",
    port=5906,
    database="TeamX_BIGAI",
    query={"charset": "utf8mb4"},
)

# db_url = URL.create(
#     drivername="mysql+pymysql",
#     username="agentictrl",
#     password="`1qaz~!QAZ",
#     host="112.125.88.107",
#     port=5906,
#     database="BIGAI",
#     query={"charset": "utf8mb4"},
# )

engine = create_engine(db_url, pool_pre_ping=True)

In [ ]:

metadata = MetaData()

# 反射全部表结构
metadata.reflect(bind=engine)

print("数据库 BIGAI 包含以下表：")
for tbl in metadata.sorted_tables:
    print(f"\n🗂  表：{tbl.name}")
    for col in tbl.columns:
        print(f"   - {col.name} ({col.type}) {'PK' if col.primary_key else ''}")

In [ ]:
run_id="results/test_for_train_pass8_gpu8_env77_20250817_1345" # results/test_for_train_instruction
with engine.connect() as conn:
    result = conn.execute(text("select * from(SELECT run_id, count(*) ct FROM `rollout_run` group by run_id) t order by ct desc"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
import pandas as pd; pd.DataFrame(rows)[5:]


In [2]:
# 查看run_id已采样的数据
run_id="results/examples_linux_osworld_0912_4" # results/test_for_train_instruction  results/pass@32_trainset90
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM `rollout_run` where run_id = 'results/singlehard_pass8_gpu2_env20_maxstep30_20250905_1616' order by create_at desc"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
import pandas as pd; df=pd.DataFrame(rows)#[['trajectory_id','task_id','reward']].to_json("data_pass@32_trainset90_0826.json", orient="records", force_ascii=False, indent=4)
# df.groupby(['task_id'], as_index=False).agg(success_rate=('reward', 'mean'),mean_step=('num_chunks', 'mean'),total=('num_chunks', 'size')).sort_values('task_id')
df.agg(reward=('reward','mean'))

rollout_run 共 7917 行


,reward
reward,0.247063


In [ ]:
import os
import re
import json
import matplotlib.pyplot as plt
from collections import defaultdict

# ====== 配置 ======
k = 5
outdir = "async_pass8_train15hard_lr1e-6_bz4_minibs64_downsample_stepwise_kl_maxstep30"
os.makedirs(outdir, exist_ok=True)

# 合并脚本输出的 JSON：{"0": [...task_ids], "1":[...], ...}
STEP_TASK_JSON = "trainset15hard_pass8_gpu2_env20_maxstep30_20250901_1835_merged_task_ids.json"

# ====== 读取 step_num -> task_ids，并构造 task_id -> set(step_num) ======
with open(STEP_TASK_JSON, "r", encoding="utf-8") as f:
    step2tasks_raw = json.load(f)

# keys 转 int，values 转为 set，随后再倒排
step2tasks = {int(s): set(map(str, tids)) for s, tids in step2tasks_raw.items()}
task2steps = defaultdict(set)
for s, tids in step2tasks.items():
    for t in tids:
        task2steps[str(t)].add(s)

# ====== 1) 从 model_version 提取 step_num；不匹配格式的一律记为 0（初始模型） ======
pattern = re.compile(r'global_step_(\d+)')

df2 = df.copy()
df2['step_num'] = (
    df2['model_version'].astype(str)
       .str.extract(pattern)      # 提取到的是字符串或 NaN
       .fillna('0')               # 不匹配的设为 '0'
       .astype(int)               # 转成整数
)
# 确保 task_id 为字符串，和合并 JSON 中一致
df2['task_id'] = df2['task_id'].astype(str)

# ====== 2) 按 task_id + step_num 分组，计算成功率（mean(reward)） ======
df_task_step = (
    df2.groupby(['task_id', 'step_num'], as_index=False)
       .agg(success_rate=('reward', 'mean'))
)

# ====== 3) 每 k 个 task_id 画一张折线图并保存；出现的 step 点放大 ======
task_ids = sorted(df_task_step['task_id'].unique().tolist())

# 可调参数：普通点大小、放大点大小
normal_ms = 36    # 相当于 markersize ~ 6 （scatter 的 s 单位是点^2）
highlight_ms = 160

saved_files = []
for i in range(0, len(task_ids), k):
    chunk = task_ids[i:i+k]

    plt.figure(figsize=(15, 6))
    for tid in chunk:
        g = (df_task_step[df_task_step['task_id'] == tid]
             .sort_values('step_num'))

        # 先画折线（带小点）
        line, = plt.plot(
            g['step_num'], g['success_rate'],
            marker='o', linestyle='-',
            label=str(tid), linewidth=1.6, markersize=4
        )
        color = line.get_color()

        # 需要放大的 step：task2steps 中出现的 step
        steps_hi = task2steps.get(str(tid), set())
        if steps_hi:
            m = g['step_num'].isin(steps_hi)
            # 基础小点（与 plot 的小点视觉上等价）可选：如果你想更细控制，可先去掉 plot 的 marker
            # 然后用下面两段分别画普通点与高亮点

            # （可选）重画普通点，保证普通点与高亮点大小对比更明显
            # m_norm = ~m
            # plt.scatter(g.loc[m_norm, 'step_num'], g.loc[m_norm, 'success_rate'],
            #             s=normal_ms, c=[color], edgecolors='none', zorder=2)

            # 叠加高亮（放大）点
            plt.scatter(
                g.loc[m, 'step_num'], g.loc[m, 'success_rate'],
                s=highlight_ms, c=[color],
                edgecolors='black', linewidths=0.8, zorder=3
            )

    plt.xlabel('step_num')
    plt.ylabel('success rate (mean reward)')
    plt.title(f'Success rate vs. step_num | tasks {i+1}-{i+len(chunk)}')
    plt.legend(title='task_id', loc='best', frameon=True)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    fname = os.path.join(outdir, f'success_rate_tasks_{i+1:03d}_{i+len(chunk):03d}.png')
    plt.savefig(fname, dpi=150)
    plt.close()
    saved_files.append(fname)

print(f"Saved {len(saved_files)} figures to '{outdir}':")
for f in saved_files:
    print(' -', f)

In [ ]:
# 查看run_id已采样的数据
run_id="results/trainset15_pass16_gpu2_env20_maxstep15_20250829_1712" # results/test_for_train_instruction  results/pass@32_trainset90
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM `rollout_run` where run_id = 'results/pass@32_trainset90_0826' order by create_at desc"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
import pandas as pd; df=pd.DataFrame(rows)#[['trajectory_id','task_id','reward']].to_json("data_pass@32_trainset90_0826.json", orient="records", force_ascii=False, indent=4)
df[(df['task_id']=='82279c77-8fc6-46f6-9622-3ba96f61b477')&(df['num_chunks']==14)]['trajectory_id'].iloc[0]

In [ ]:
df_task = (
    df.groupby('task_id', as_index=False)
      .agg(mean_reward=('reward', 'mean'),
           mean_num_chunks=('num_chunks', 'mean'),
           min_num_chunks=('num_chunks', 'min'))
)
df_task_filtered = df_task.query('0 < mean_reward < 0.3').query('25 < mean_num_chunks < 50')
df_task_filtered.sort_values('mean_num_chunks', ignore_index=True)

In [ ]:
import numpy as np
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM `rollout_run` where run_id='results/pass@32_trainset90_0826' order by create_at desc"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")

df = pd.DataFrame(rows)

g = (
    df.groupby("task_id", dropna=False)["reward"]
      .agg(cnt="size", mean_reward="mean")
)
valid_tasks = g.index[(g["cnt"] == 32) & (g["mean_reward"].gt(0.4) & g["mean_reward"].lt(0.6))]
filtered = df[df["task_id"].isin(valid_tasks)].copy()

# 只保留需要导出的列（先带上原始 num_chunks，后面会覆盖）
cols = ["trajectory_id", "task_id", "reward", "num_chunks"]
out = filtered[cols].copy()
out["num_chunks"] = np.ceil(out["num_chunks"] / 5).astype(int)
out["reward"] = out["reward"].astype(float)
out = out.dropna(subset=["trajectory_id", "task_id", "reward", "num_chunks"])
out.to_json("trajectories.json", orient="records", force_ascii=False, indent=4)

In [ ]:
# 查看run_id已保存的checkpoints
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM `checkpoint` where run_id = 'results/pass8_20250904_train15_pass8_gpu2_env20_vllm_logp_maxstep15_tesl_vllm_logp_test3' order by created_at"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"checkpoint 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
print(rows[-1]['path'])
import pandas as pd; pd.DataFrame(rows)

In [ ]:

# 从rollout_run中删除某run_id的数据
run_id="results/pass8_20250904_train15_pass8_gpu2_env20_vllm_logp_maxstep15_tesl_vllm_logp_test3"
with engine.begin() as conn:
    will = conn.execute(
        text("SELECT COUNT(*) FROM `checkpoint` WHERE `run_id` = :rid"),
        {"rid": run_id}
    ).scalar_one()
    print(f"将删除 {will} 行（run_id = {run_id}）")

    res = conn.execute(
        text("DELETE FROM `checkpoint` WHERE `run_id` = :rid"),
        {"rid": run_id}
    )
    print(f"✅ 已删除 {res.rowcount} 行")

In [ ]:

# 从rollout_run中删除某run_id的数据
run_id="results/trainset15_pass8_gpu2_env20_maxstep30_20250901_1326"
with engine.begin() as conn:
    will = conn.execute(
        text("SELECT COUNT(*) FROM `rollout_run` WHERE `run_id` = :rid"),
        {"rid": run_id}
    ).scalar_one()
    print(f"将删除 {will} 行（run_id = {run_id}）")

    res = conn.execute(
        text("DELETE FROM `rollout_run` WHERE `run_id` = :rid"),
        {"rid": run_id}
    )
    print(f"✅ 已删除 {res.rowcount} 行")

In [ ]:
run_id="results/test_for_train_pass8_gpu8_env77_20250817_1345"
with engine.connect() as conn:
    cnt = conn.execute(
        text("""
            SELECT COUNT(*) FROM `rollout_run`
            WHERE `run_id` = :rid AND `used` != 0
        """),
        {"rid": run_id},
    ).scalar_one()
print(f"确认 used!=0 的行数：{cnt}")

with engine.begin() as conn:
    res = conn.execute(
        text("""
            UPDATE `rollout_run`
            SET `used` = 0
            WHERE `run_id` = :rid
        """),
        {"rid": run_id},
    )
    print(f"✅ 更新行数: {res.rowcount}")

In [ ]:
# 更新used = 0 数据的model version
run_id="results/pass@32_trainset90"
model_version = "/workspace/computer-use/verl/checkpoints/verl_osworld_grpo/osworld_all_feasible_reward_script_grpo_k8s_0813_h8zdohoq/global_step_10/actor/huggingface"
model_version = "/capacity/userdata/vcfenxd75jiv/shichenrui/ui_tars/ByteDance-Seed/UI-TARS-1.5"
with engine.connect() as conn:
    cnt, sample = conn.execute(
        text("""
            SELECT COUNT(*) AS cnt, MAX(`model_version`) AS sample_version
            FROM `rollout_run`
            WHERE `run_id` = :rid AND `used` = 0
        """),
        {"rid": run_id},
    ).one()
print(f"used=0 的剩余行数: {cnt}，示例版本: {sample}")

with engine.begin() as conn:
    res = conn.execute(
        text("""
            UPDATE `rollout_run`
            SET `model_version` = :mv
            WHERE `run_id` = :rid AND `used` = 0
        """),
        {"mv": model_version, "rid": run_id},
    )
    print(f"✅ 更新行数: {res.rowcount}")

In [ ]:
# 统计run_id validation 结果

import numpy as np
import json
import pandas as pd

# run_ids = ["results/val_train150_uitars_maxstep50", 
#            "results/val_trainset90_px_08221633_step10",
#            "results/val_trainset90_px_08221633_step20", ]
run_ids = ["results/val_train154_maxstep30_tmp0_uitars", 
           "results/val_train150_uitars_maxstep50",
           #"results/val_train154_maxstep30_tmp07_uitars", 
           #"results/val_train154_maxstep30_tmp07_20250829_1122_step36",
           "results/val_train154_maxstep30_tmp0_20250829_1122_step36",
           #results/val_train154_maxstep30_tmp07_20250830_1230_step120",
           "results/val_train154_maxstep30_tmp0_20250830_1230_step120",
           "results/val_train154_maxstep30_tmp0_20250830_1230_step130",
           "results/val_train154_maxstep100_tmp0_20250830_1230_step130"]

with open("../evaluation_examples/test_trainset_90.json", "r", encoding="utf-8") as f:
    mapping = json.load(f)
eval_task_ids = set()
for _, lst in mapping.items():
    eval_task_ids.update(lst)
    
placeholders = ", ".join([f":id{i}" for i in range(len(run_ids))]) or ":id0"
sql = text(f"""
    SELECT reward, run_id, task_id
    FROM rollout_run
    WHERE run_id IN ({placeholders})
""")
params = {f"id{i}": rid for i, rid in enumerate(run_ids)} or {"id0": None}

with engine.begin() as conn:
    df = pd.read_sql(sql, conn, params=params)

def agg_by_run_id(df: pd.DataFrame) -> pd.DataFrame:
    x = df["reward"].fillna(0)
    reward_non_neg1 = np.where(x.eq(-1), 0, x).astype(float)

    df_calc = pd.DataFrame({
        "run_id": df["run_id"].values,
        "reward_non_neg1": reward_non_neg1
    })

    total_rows = df_calc.groupby("run_id").size().rename("total_rows")
    total_score = df_calc.groupby("run_id")["reward_non_neg1"].sum().rename("total_score")

    result = (
        pd.concat([total_rows, total_score], axis=1)
        .assign(success_rate=lambda d: d["total_score"] / d["total_rows"])
        .reset_index()
        .sort_values("run_id")
    )
    return result

#df_reward0 = df[df["task_id"].isin(task_reward0)]

df_subset = df[df["task_id"].isin(eval_task_ids)]
result_subset = agg_by_run_id(df).rename(columns={
    "metric": "metric_subset",
    "total_score": "total_score",
    "total_rows": "total_rows_subset",
})

result_subset

In [ ]:
len(eval_task_ids)

In [ ]:
result_subset = agg_by_run_id(df_subset).rename(columns={
    "metric": "metric_subset",
    "total_score": "total_score",
    "total_rows": "total_rows_subset",
})
result_subset